# Installation tips

* Create Anaconda virtual environment with ipython notebook support
  > conda create -n tf ipython-notebook --yes
* The set up as explained in the [official site](https://www.tensorflow.org/versions/r0.9/get_started/os_setup.html#anaconda-installation) failed for me. Something to do with failure to update setup tools. The remedy was doing as explained in [here](https://github.com/ContinuumIO/anaconda-issues/issues/542): 
 > pip install --ignore-installed --upgrade pip setuptools

# Hellow TensorFlow

For the [O'reilly post](https://www.oreilly.com/learning/hello-tensorflow)

In [1]:
import tensorflow as tf

In [61]:
from tensorflow.python.framework import ops

ops.reset_default_graph()
graph = tf.get_default_graph()
graph.get_operations()

# Model of a simple neuron: y <-- x * w
x       = tf.constant(1.0,name='x')
w       = tf.Variable(0.8,name='w')
y       = tf.mul(w , x, name='y')

y_     = tf.constant(0.0,name='y_train')
loss   = (y-y_)**2

#--------------------------------------------------------------
# Print the nodes of teh graph, also called 'operations' or 'ops'
#--------------------------------------------------------------
print 'Operations in graph \n==========================='
for op in graph.get_operations():
    print op.name

#--------------------------------------------------------------
# Get the last operation (**2) 
#--------------------------------------------------------------
op = graph.get_operations()[-1]
print '\nLaste operation in graph is : ',op.name
print '\nInputs:\n===========  '
for op_input in op.inputs: print '  ',op_input
print '\nOutputs:\n===========  '
for op_output in op.outputs: print '  ',op_output
    

#--------------------------------------------------------------
# Define gradient descent optimizer
#--------------------------------------------------------------
optim          = tf.train.GradientDescentOptimizer(learning_rate=0.025)
grads_and_vars = optim.compute_gradients(loss)

#--------------------------------------------------------------
# NOdes of graph after all operations related to GD are added
#--------------------------------------------------------------
print '\nOperations in graph \n==========================='
for op in graph.get_operations():
    print op.name


#--------------------------------------------------------------
# Get the last operation and investigate it
#--------------------------------------------------------------
op = graph.get_operations()[-1]
print '\nLaste operation in graph is : ',op.name
print '\nInputs:\n===========  '
for op_input in op.inputs: print '  ',op_input
print '\nOutputs:\n===========  '
for op_output in op.outputs: print '  ',op_output

#--------------------------------------------------------------
# Create an operation that initializes all variables in the graph
#--------------------------------------------------------------
init = tf.initialize_all_variables()


op = graph.get_operations()[-1]
print '\nLaste operation in graph is now: ',op.name

#--------------------------------------------------------------
#--------------------------------------------------------------
# Running the graph
#--------------------------------------------------------------
#--------------------------------------------------------------
print '\nCreate a session to evaluate graph operations\n==============================================='    
sess = tf.Session()                      # For regular python code
#sess  = tf.InteractiveSession(graph=graph) # for ipython notebook

sess.run(init)
print 'Run session to obtaine the value of x: ',sess.run(x)
print 'Run session to obtaine the value of y: ',sess.run(y)
print 'Initial value of loss function       : ',sess.run(loss)


print '\nCompute gradients:                   ', sess.run(grads_and_vars[0][0])

sess.run(optim.apply_gradients(grads_and_vars))
print '\nUpdates weights after one BackProp step: {0}, loss function={1}'.format(sess.run(w),sess.run(loss))

#--------------------------------------------------------------
# Full training step: compute gradients and apply them
#--------------------------------------------------------------
train_step = tf.train.GradientDescentOptimizer(0.025).minimize(loss)
print '\n'
for i in range(200):
    sess.run(train_step)
    if not i%10:
        print '{0:4} : Weight={1:5.3}, loss={2:6.4}'.format(i,sess.run(w),sess.run(loss))


ops.reset_default_graph()

Operations in graph 
x
w/initial_value
w
w/Assign
w/read
y
y_train
sub
pow/y
pow

Laste operation in graph is :  pow

Inputs:
   Tensor("sub:0", shape=(), dtype=float32)
   Tensor("pow/y:0", shape=(), dtype=float32)

Outputs:
   Tensor("pow:0", shape=(), dtype=float32)

Operations in graph 
x
w/initial_value
w
w/Assign
w/read
y
y_train
sub
pow/y
pow
gradients/Shape
gradients/Const
gradients/Fill
gradients/pow_grad/Shape
gradients/pow_grad/Shape_1
gradients/pow_grad/BroadcastGradientArgs
gradients/pow_grad/mul
gradients/pow_grad/sub/y
gradients/pow_grad/sub
gradients/pow_grad/Pow
gradients/pow_grad/mul_1
gradients/pow_grad/Sum
gradients/pow_grad/Reshape
gradients/pow_grad/mul_2
gradients/pow_grad/Log
gradients/pow_grad/mul_3
gradients/pow_grad/Sum_1
gradients/pow_grad/Reshape_1
gradients/pow_grad/tuple/group_deps
gradients/pow_grad/tuple/control_dependency
gradients/pow_grad/tuple/control_dependency_1
gradients/sub_grad/Shape
gradients/sub_grad/Shape_1
gradients/sub_grad/BroadcastGradie

# Training and visualization

To  see the graphs invoke the command:
> tensorboard --logdir=log_simple_stat

which can then be viewed in the browser at
> localhost:6006/#events


In [62]:
import tensorflow as tf

x = tf.constant(1.0, name='input')
w = tf.Variable(0.8, name='weight')
y = tf.mul(w, x, name='output')
y_ = tf.constant(0.0, name='correct_value')
loss = tf.pow(y - y_, 2, name='loss')
train_step = tf.train.GradientDescentOptimizer(0.025).minimize(loss)

for value in [x, w, y, y_, loss]:
    tf.scalar_summary(value.op.name, value)

summaries = tf.merge_all_summaries()

sess = tf.Session()
summary_writer = tf.train.SummaryWriter('log_simple_stats', sess.graph)

sess.run(tf.initialize_all_variables())
for i in range(100):
    summary_writer.add_summary(sess.run(summaries), i)
    sess.run(train_step)
